trying to recreate:

https://www.kaggle.com/algila/inception-v3-and-k-fold-in-python-0-98996

In [1]:
import os, glob, bcolz, gc

import numpy as np
import pandas as pd

from tqdm import tqdm
from scipy import ndimage, misc

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score

from keras import backend as K
from keras import optimizers
from keras.models import Model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator, random_rotation, random_shear, random_zoom, random_shift, flip_axis

from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers.convolutional import MaxPooling2D, Convolution2D
from keras.layers.advanced_activations import PReLU, LeakyReLU

from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.inception_v3 import preprocess_input as preprocess_input_incep_xcep
from keras.applications.imagenet_utils import preprocess_input as preprocess_input_vgg_resnet

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [8]:
def read_img(img_path, img_shape):
    img = misc.imread(img_path)
    img = misc.imresize(img, img_shape)
    return img

def read_imgs(img_height, img_width):
    train_img, test_img = [],[]
    for img_path in tqdm(train_set['name'].iloc[:]):
        train_img.append(read_img(os.path.join(path, 'train', str(img_path)+'.jpg'), (img_height, img_width)))

    for img_path in tqdm(test_set['name'].iloc[:]):
        test_img.append(read_img(os.path.join(path, 'test', str(img_path)+'.jpg'), (img_height, img_width)))
    return np.array(train_img), np.array(test_img)

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]

def freeze_model(model):
    for layer in model.layers:
        layer.trainable = False
    return model

def grab_optimizer(opt, lr):
    if opt == 'sgd':
        return optimizers.SGD(lr=lr, decay=1e-6, momentum=0.8, nesterov=True)
    elif opt == 'adam':
        return optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    elif opt == 'adagrad':
        return optimizers.Adagrad(lr=lr)
    elif opt == 'rmsprop':
        return optimizers.RMSprop(lr=lr)
    
def dense_block(units, activation, drop_prob, inputs):
    x = Dense(units, activation=None)(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Dropout(drop_prob)(x)
    return x

def make_incepv3_conv(input_shape):
    base_model = InceptionV3(input_shape=input_shape, weights='imagenet', include_top=False)
    base_model = freeze_model(base_model)
    return base_model

def make_ft_dense(input_shape, optimizer):
    inputs = Input(shape=input_shape)
    m = Flatten()(inputs)
    m = dense_block(1024, 'relu', 0.5, inputs=m)
    m = dense_block(1024, 'relu', 0.5, inputs=m)
    outputs = dense_block(1, 'sigmoid', 0, inputs=m)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def make_incepv3_ft(input_shape, optimizer):
    base_model = InceptionV3(input_shape=input_shape, weights='imagenet', include_top=False, pooling='avg')
    base_model = freeze_model(base_model)
    m = dense_block(1024, 'relu', 0.6, inputs=base_model.layers[-1].output)
    outputs = dense_block(1, 'sigmoid', 0, inputs=m)
    
    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# does not use precomputiation so it can use data augmentation
def train_kfolds_ft_conv(model, base_model, train_data, train_label, augment, model_out, model_init_weights, epochs, kfolds, batch_size):
    kf = KFold(n_splits=kfolds, shuffle=True)
    
    i = 0
    models_stats = {}
    for train_ixs, valid_ixs in kf.split(train_data):
        x_train = train_data[train_ixs]
        x_valid = train_data[valid_ixs]
        y_train = train_label[train_ixs]
        y_valid = train_label[valid_ixs]
        
        #augment the data
        if augment:
            for i in range(len(x_train)):
                x_train[i] = random_rotation(x_train[i], 20) 
                x_train[i] = random_shear(x_train[i], 0.2)
                x_train[i] = random_zoom(x_train[i], [0.8, 1.2])
                x_train[i] = random_shift(x_train[i], 0.2, 0.2)
        
        # precompute features
        x_train = base_model.predict(x_train)
        x_valid = base_model.predict(x_valid)
                
        #re-initialzie the weights of the model on each run
        #by loading thi intiial stored weights from file
        model.load_weights(model_init_weights)
        model_out_file = '{}_{}.model'.format(model_out, str(i))
        model_checkpoint = ModelCheckpoint(model_out_file, 
                                            monitor='val_loss', 
                                            save_best_only=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_acc',
                              patience=5,
                              verbose=1,
                              factor=0.1,
                              cooldown=10,
                              min_lr=0.00001)
        
        model.fit(x=x_train, y=y_train, 
                      batch_size=batch_size,
                      validation_data=(x_valid,y_valid),
                      epochs=epochs,
                      verbose=1,
                      callbacks=[model_checkpoint, reduce_lr])
        
        model.load_weights(model_out_file)
        eval_tr = model.evaluate(x_train, y_train)
        eval_va = model.evaluate(x_valid, y_valid)
        tr_score = roc_auc_score(y_train, model.predict(x_train)[:, 0])
        va_score = roc_auc_score(y_valid, model.predict(x_valid)[:, 0])
        
        print('\n')
        print('kfold: {}'.format(str(i)))
        print('best model train acc: {}, loss: {}'.format(eval_tr[1], eval_tr[0]))
        print('best model valid acc: {}, loss: {}'.format(eval_va[1], eval_va[0]))
        print('best model train aroc score: {}, valid aroc score: {}'.format(tr_score, va_score))
        print('\n')
        models_stats[model_out_file] = {'score_tr_va':[tr_score, va_score], 'train_acc_loss':[eval_tr[1], eval_tr[0]], 'val_acc_loss':[eval_va[1], eval_va[0]]}
        
        with open(os.path.join(models_path,'{}_{}.out'.format(model_out,'history')), 'a') as f:
            f.write('kfold: {}'.format(str(i)))
            f.write('best model train acc: {}, loss: {}'.format(eval_tr[1], eval_tr[0]))
            f.write('best model valid acc: {}, loss: {}'.format(eval_va[1], eval_va[0]))
            f.write('best model train aroc score: {}, valid aroc score: {}'.format(tr_score, va_score))
            f.write('\n')
        
        i += 1
    
    return models_stats

In [9]:
DATA_DIR = os.path.join('/scratch', 'yns207', 'data_invasive')

path = DATA_DIR
test_path = os.path.join(path, 'test')
models_path = os.path.join(path, 'results')
train_path = os.path.join(path, 'train')
valid_path = os.path.join(path, 'valid')

In [10]:
%cd $path
train_img = load_array('aug_3_train_img.dat')
hold_img = load_array('aug_3_hold_img.dat')
train_labels = load_array('aug_3_train_labels.dat')
hold_labels = load_array('aug_3_hold_labels.dat')

/scratch/yns207/data_invasive


In [11]:
# put training and holdout data together again
train_img = np.concatenate([train_img, hold_img])
train_labels = np.concatenate([train_labels, hold_labels])

In [13]:
%cd $path
batch_size = 64
epochs = 30
kfolds = 5
lr = 0.001

model_name = 'invasive_inceptionv3_lbrep_aug7'
init_weights_model = '{}_base.model'.format(model_name)

# create model and save initial weights
base_model = make_incepv3_conv(train_img[0].shape)

dense_model = make_ft_dense(input_shape=tuple(base_model.output[0].shape.as_list()), optimizer=grab_optimizer('adam', lr))
dense_model.save_weights(init_weights_model)

train_img_proc = preprocess_input_incep_xcep(train_img.astype(np.float32))

# train dense model on folds
performance = train_kfolds_ft_conv(dense_model, base_model, train_img_proc, train_labels, True, model_name, init_weights_model, epochs, kfolds, batch_size)

/scratch/yns207/data_invasive


KeyboardInterrupt: 

In [14]:
%cd $path
batch_size = 64
epochs = 30
kfolds = 5
lr = 0.001

model_name = 'invasive_inceptionv3_lbrep_aug7'
init_weights_model = '{}_base.model'.format(model_name)

# create model and save initial weights
base_model = make_incepv3_conv(train_img[0].shape)

dense_model = make_ft_dense(input_shape=tuple(base_model.output[0].shape.as_list()), optimizer=grab_optimizer('adam', lr))
dense_model.save_weights(init_weights_model)

train_img_proc = preprocess_input_incep_xcep(train_img.astype(np.float32))

# train dense model on folds
performance = train_kfolds_ft_conv(dense_model, base_model, train_img_proc, train_labels, False, model_name, init_weights_model, epochs, kfolds, batch_size)

/scratch/yns207/data_invasive
Train on 1836 samples, validate on 459 samples
Epoch 1/30
1836/1836 [==============================] - 13s - loss: 0.4339 - acc: 0.8867 - val_loss: 0.2657 - val_acc: 0.9063

In [ ]:
# dont have much to lose ensembling this...


# replication attempt below

In [ ]:
train_set = pd.read_csv(os.path.join(path, 'train_labels.csv'))
test_set = pd.read_csv(os.path.join(path, 'sample_submission.csv'))

def read_img(img_path, img_shape):
    img = misc.imread(img_path)
    img = misc.imresize(img, img_shape)
    return img

def read_imgs(img_height, img_width):
    train_img, test_img = [],[]
    for img_path in tqdm(train_set['name'].iloc[:]):
        train_img.append(read_img(os.path.join(path, 'train', str(img_path)+'.jpg'), (img_height, img_width)))

    for img_path in tqdm(test_set['name'].iloc[:]):
        test_img.append(read_img(os.path.join(path, 'test', str(img_path)+'.jpg'), (img_height, img_width)))
    return np.array(train_img), np.array(test_img)

train_img, test_img = read_imgs(224,224)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output


import cv2
import os, gc, sys, glob
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn import model_selection
from sklearn import metrics


import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model, load_model
from keras import applications
from keras.callbacks import ReduceLROnPlateau

from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Transfer learning da Inception V3 traino solo gli ultimi fully connected layers
img_rows, img_cols, img_channel = 224, 224, 3
base_model = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet',pooling='avg', input_shape=(img_rows, img_cols, img_channel))

#Adding custom Layers
add_model = Sequential()
add_model.add(Dense(1024, activation='relu',input_shape=base_model.output_shape[1:]))
add_model.add(Dropout(0.60))
add_model.add(Dense(1, activation='sigmoid'))
print(add_model.summary())

# creating the final model
model = Model(inputs=base_model.input, outputs=add_model(base_model.output))

# compile the model
opt = optimizers.adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#opt = optimizers.SGD(lr = 0.0001, momentum = 0.8, nesterov = True)
# meglio la funzione sotto che lega il learning rate decay al monitor che si vuole
reduce_lr = ReduceLROnPlateau(monitor='val_acc',
                              patience=5,
                              verbose=1,
                              factor=0.1,
                              cooldown=10,
                              min_lr=0.00001)  # funzione di kesar to reduce learning rate (new_lr = lr*factor)
model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics=['accuracy'])

n_fold = 5
kf = model_selection.KFold(n_splits = n_fold, shuffle = True)
eval_fun = metrics.roc_auc_score

def run_oof(tr_x, tr_y, te_x, kf):
    preds_train = np.zeros(len(tr_x), dtype = np.float)
    preds_test = np.zeros(len(te_x), dtype = np.float)
    train_loss = []; test_loss = []

    i = 1
    for train_index, test_index in kf.split(tr_x):
        x_tr = tr_x[train_index]; x_te = tr_x[test_index]
        y_tr = tr_y[train_index]; y_te = tr_y[test_index]

        datagen = ImageDataGenerator(
            # featurewise_center = True,
            rotation_range = 20,
            width_shift_range = 0.2,
            height_shift_range = 0.2,
            # zca_whitening = True,
            shear_range = 0.2,
            zoom_range = 0.2,
            horizontal_flip = True,
            vertical_flip = True,
            fill_mode = 'nearest')
        datagen.fit(x_tr)

        model.fit_generator(datagen.flow(x_tr, y_tr, batch_size = 64),
            validation_data = (x_te, y_te), callbacks=[reduce_lr],
            steps_per_epoch = len(train_img) / 64, epochs = 45, verbose = 2)

        train_loss.append(eval_fun(y_tr, model.predict(x_tr)[:, 0]))
        test_loss.append(eval_fun(y_te, model.predict(x_te)[:, 0]))

        preds_train[test_index] = model.predict(x_te)[:, 0]
        preds_test += model.predict(te_x)[:, 0]

        print('{0}: Train {1:0.5f} Val {2:0.5f}'.format(i, train_loss[-1], test_loss[-1]))
        i += 1

    print('Train: ', train_loss)
    print('Val: ', test_loss)
    print('Train{0:0.5f}_Test{1:0.5f}\n\n'.format(np.mean(train_loss), np.mean(test_loss)))
    preds_test /= n_fold
    return preds_train, preds_test

train_pred, test_pred = run_oof(train_img, train_labels, test_img, kf)

test_set['invasive'] = test_pred
test_set.to_csv('../submit.csv', index = None)